# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [325]:
accident_df = pd.read_csv('data/AviationData.csv', encoding='cp1252')

/tmp/ipykernel_690271/2675841452.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_df = pd.read_csv('data/AviationData.csv', encoding='cp1252')


In [326]:
accident_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [327]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [328]:
accident_df.shape

(88889, 31)

In [329]:
accident_df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [330]:
# Only airplanes that are professional make/model and could potentially still be active
# Assume 40-year lifespan for planes

# Look more closely at data from what seem like potentially relevant columns
accident_df[['Event.Date', 'Aircraft.Category', 'Make', 'Model', 'Amateur.Built', 'FAR.Description', 'Report.Status', 'Publication.Date']].head(10)

,Event.Date,Aircraft.Category,Make,Model,Amateur.Built,FAR.Description,Report.Status,Publication.Date
0,1948-10-24,NaN,Stinson,108-3,No,NaN,Probable Cause,NaN
1,1962-07-19,NaN,Piper,PA24-180,No,NaN,Probable Cause,19-09-1996
2,1974-08-30,NaN,Cessna,172M,No,NaN,Probable Cause,26-02-2007
3,1977-06-19,NaN,Rockwell,112,No,NaN,Probable Cause,12-09-2000
4,1979-08-02,NaN,Cessna,501,No,NaN,Probable Cause,16-04-1980
5,1979-09-17,Airplane,Mcdonnell Douglas,DC9,No,Part 129: Foreign,Probable Cause,19-09-2017
6,1981-08-01,NaN,Cessna,180,No,NaN,Probable Cause,06-11-2001
7,1982-01-01,Airplane,Cessna,140,No,Part 91: General Aviation,Probable Cause,01-01-1982
8,1982-01-01,Airplane,Cessna,401B,No,Part 91: General Aviation,Probable Cause,01-01-1982
9,1982-01-01,NaN,North American,NAVION L-17B,No,NaN,Probable Cause,01-01-1982


In [331]:
accident_df['Amateur.Built'].value_counts()

Amateur.Built
No     80312
Yes     8475
Name: count, dtype: int64

In [332]:
accident_df['Event_Date'] = pd.to_datetime(accident_df['Event.Date'], format='%Y-%m-%d')

In [333]:
accident_df_filter_plane = accident_df.loc[(accident_df['Amateur.Built'] != 'Yes') & 
                                       (accident_df['Event_Date'].dt.year >= 1985)]

In [334]:
accident_df_filter_plane.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70564 entries, 10613 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                70564 non-null  object        
 1   Investigation.Type      70564 non-null  object        
 2   Accident.Number         70564 non-null  object        
 3   Event.Date              70564 non-null  object        
 4   Location                70517 non-null  object        
 5   Country                 70389 non-null  object        
 6   Latitude                30196 non-null  object        
 7   Longitude               30190 non-null  object        
 8   Airport.Code            40031 non-null  object        
 9   Airport.Name            41418 non-null  object        
 10  Injury.Severity         69565 non-null  object        
 11  Aircraft.damage         67601 non-null  object        
 12  Aircraft.Category       25403 non-null  object 

In [335]:
accident_df_filter_plane.shape

(70564, 32)

In [336]:
accident_df_filter_plane.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Event_Date
10613,20001214X35598,Accident,NYC85FNC02,1985-01-01,"HOPKINTON, NH",United States,NaN,NaN,LEB,"LEBANON, NH",...,NaN,0.0,2.0,0.0,0.0,IMC,Approach,Probable Cause,NaN,1985-01-01
10614,20001214X35495,Accident,DEN85FA055,1985-01-01,"EDGEWOOD, NM",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,NaN,1985-01-01
10615,20001214X35494,Accident,DCA85RA007,1985-01-01,"LA PAZ, Bolivia",Bolivia,NaN,NaN,LPB,LA PAZ INTERNATIONAL,...,NaN,29.0,0.0,0.0,0.0,UNK,NaN,Foreign,06-02-1995,1985-01-01
10616,20001214X35566,Accident,MIA85FA071,1985-01-01,"ODESSA, FL",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,2.0,2.0,0.0,VMC,Maneuvering,Probable Cause,NaN,1985-01-01
10617,20001214X35463,Accident,ATL85LA075,1985-01-01,"DUBLIN, NC",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,1.0,VMC,Cruise,Probable Cause,NaN,1985-01-01


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [337]:
# Review what injury-related columns look like
accident_df_filter_plane[['Injury.Severity','Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].tail(10)

,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
88879,NaN,0.0,0.0,0.0,0.0
88880,Non-Fatal,0.0,0.0,0.0,3.0
88881,Minor,0.0,1.0,0.0,0.0
88882,Minor,0.0,1.0,0.0,1.0
88883,Fatal,1.0,0.0,0.0,0.0
88884,Minor,0.0,1.0,0.0,0.0
88885,NaN,0.0,0.0,0.0,0.0
88886,Non-Fatal,0.0,0.0,0.0,1.0
88887,NaN,0.0,0.0,0.0,0.0
88888,Minor,0.0,1.0,0.0,1.0


In [338]:
accident_df_filter_plane[['Injury.Severity','Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].isna().sum()

Injury.Severity             999
Total.Fatal.Injuries      10173
Total.Serious.Injuries    11112
Total.Minor.Injuries      10586
Total.Uninjured            4931
dtype: int64

In [339]:
# There are a sizeable number of null values in the passenger counts, which will affect the later total passenger calculation
# While we don't know if these are meant to be 0 or are simply missing values, removing them would be removing more than 10% of the total dataset
# Instead, these null values will be set to 0 so that only rows with 0 passengers total will be filtered
accident_df_filter_injury = accident_df_filter_plane.fillna({'Total.Fatal.Injuries': 0, 'Total.Serious.Injuries': 0,
                                                     'Total.Minor.Injuries': 0, 'Total.Uninjured': 0})
accident_df_filter_injury[['Injury.Severity','Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].isna().sum()

Injury.Severity           999
Total.Fatal.Injuries        0
Total.Serious.Injuries      0
Total.Minor.Injuries        0
Total.Uninjured             0
dtype: int64

In [340]:
accident_df_filter_injury['Total_Passengers'] = accident_df_filter_injury['Total.Fatal.Injuries'] + accident_df_filter_injury['Total.Serious.Injuries'] + \
                                           accident_df_filter_injury['Total.Minor.Injuries'] + accident_df_filter_injury['Total.Uninjured']

accident_df_filter_injury['Severe_Injury_Rate'] = (accident_df_filter_injury['Total.Fatal.Injuries'] + accident_df_filter_injury['Total.Serious.Injuries'])/ \
                                              accident_df_filter_injury['Total_Passengers']

In [341]:
accident_df_filter_injury[['Aircraft.damage', 'Injury.Severity','Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Total_Passengers', 'Severe_Injury_Rate']].tail(10)

,Aircraft.damage,Injury.Severity,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Total_Passengers,Severe_Injury_Rate
88879,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
88880,NaN,Non-Fatal,0.0,0.0,0.0,3.0,3.0,0.0
88881,NaN,Minor,0.0,1.0,0.0,0.0,1.0,1.0
88882,NaN,Minor,0.0,1.0,0.0,1.0,2.0,0.5
88883,NaN,Fatal,1.0,0.0,0.0,0.0,1.0,1.0
88884,NaN,Minor,0.0,1.0,0.0,0.0,1.0,1.0
88885,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
88886,Substantial,Non-Fatal,0.0,0.0,0.0,1.0,1.0,0.0
88887,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
88888,NaN,Minor,0.0,1.0,0.0,1.0,2.0,0.5


In [342]:
accident_df_filter_injury['Severe_Injury_Rate'].isna().sum()

np.int64(1296)

In [343]:
# Because there is still some information on the state of the plane even when we are missing passenger info, we will keep these rows
# They will need to be filtered during analysis when looking at survival/injury rates
accident_df_filter_injury.loc[accident_df_filter_injury['Severe_Injury_Rate'].isna(), 'Aircraft.damage'].unique()

array(['Substantial', 'Destroyed', 'Minor', nan, 'Unknown'], dtype=object)

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [344]:
accident_df_filter_injury['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)

In [345]:
accident_df_filter_injury['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    51352
Destroyed      13763
Minor           2370
Unknown          116
Name: count, dtype: int64

In [346]:
# Null is equivalent to unknown outcome
accident_df_filter_damage = accident_df_filter_injury.fillna({'Aircraft.damage': 'Unknown'})

In [347]:
accident_df_filter_damage['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    51352
Destroyed      13763
Unknown         3079
Minor           2370
Name: count, dtype: int64

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [348]:
len(accident_df_filter_damage['Make'].unique())

2277

In [349]:
accident_df_filter_damage['Make'].value_counts()

Make
Cessna                            18360
Piper                              9916
CESSNA                             4922
Beech                              3588
PIPER                              2839
                                  ...  
SCOTT TERRY G                         1
PERYERA ARNET                         1
BALCOMB KENNETH C III                 1
AEROPRAKT MANUFACTURING SP ZOO        1
MCGLAUGHLIN RICHARD                   1
Name: count, Length: 2276, dtype: int64

In [350]:
accident_df_filter_damage['Make'] = accident_df_filter_damage['Make'].str.upper()

In [351]:
make_counts = accident_df_filter_damage['Make'].value_counts()
make_filter = make_counts[make_counts < 50].index
accident_df_filter_make = accident_df_filter_damage.loc[~(accident_df_filter_damage['Make'].isin(make_filter))]

In [352]:
len(accident_df_filter_make['Make'].unique())

88

In [353]:
accident_df_filter_make['Make'].value_counts()

Make
CESSNA                         23282
PIPER                          12755
BEECH                           4630
BOEING                          2575
BELL                            2368
                               ...  
CAMERON                           58
AMERICAN CHAMPION AIRCRAFT        54
GREAT LAKES                       50
GRUMMAN AMERICAN AVN. CORP.       50
FLIGHT DESIGN GMBH                50
Name: count, Length: 87, dtype: int64

In [354]:
# Big manufacturers are likely to have a few variations
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('AIRBUS', na=False)]['Make'].unique()

array(['AIRBUS INDUSTRIE', 'AIRBUS'], dtype=object)

In [355]:
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('BOEING', na=False)]['Make'].unique()

array(['BOEING'], dtype=object)

In [356]:
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('BELL', na=False)]['Make'].unique()

array(['BELL', 'BELLANCA'], dtype=object)

In [357]:
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('CESSNA', na=False)]['Make'].unique()

array(['CESSNA'], dtype=object)

In [358]:
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('PIPER', na=False)]['Make'].unique()

array(['PIPER'], dtype=object)

In [359]:
accident_df_filter_make[accident_df_filter_make['Make'].str.contains('BEECH', na=False)]['Make'].unique()

array(['BEECH'], dtype=object)

In [360]:
accident_df_filter_make.loc[accident_df_filter_make['Make'].str.contains('AIRBUS', na=False), 'Make'] = 'AIRBUS'

In [361]:
# This filter was added after reviewing models
accident_df_filter_make.loc[accident_df_filter_make['Make'] == 'GRUMMAN ACFT ENG COR-SCHWEIZER', 'Make'] = 'GRUMMAN-SCHWEIZER'
accident_df_filter_make.loc[accident_df_filter_make['Make'].str.startswith('GRUMMAN ', na=False), 'Make'] = 'GRUMMAN'

accident_df_filter_make.loc[accident_df_filter_make['Make'].str.startswith('BOMBARDIER ', na=False), 'Make'] = 'BOMBARDIER'

In [362]:
accident_df_filter_make['Make'].value_counts()

Make
CESSNA                        23282
PIPER                         12755
BEECH                          4630
BOEING                         2575
BELL                           2368
                              ...  
PILATUS                          59
CAMERON                          58
AMERICAN CHAMPION AIRCRAFT       54
GREAT LAKES                      50
FLIGHT DESIGN GMBH               50
Name: count, Length: 82, dtype: int64

In [363]:
accident_df_filter_make['Make'].isna().sum()

np.int64(48)

In [364]:
accident_df_filter_make.loc[accident_df_filter_make['Make'].isna(), ['Make', 'Model', 'Aircraft.damage', 'Aircraft.Category', 'Purpose.of.flight', 'Total_Passengers', 'Severe_Injury_Rate']].head()

,Make,Model,Aircraft.damage,Aircraft.Category,Purpose.of.flight,Total_Passengers,Severe_Injury_Rate
45357,NaN,A330,Unknown,Airplane,Unknown,0.0,NaN
49759,NaN,NaN,Minor,NaN,NaN,0.0,NaN
50025,NaN,NaN,Destroyed,Airplane,NaN,6.0,1.0
51022,NaN,206,Substantial,NaN,NaN,5.0,0.2
54773,NaN,NaN,Destroyed,NaN,NaN,10.0,1.0


In [365]:
accident_df_filter_make = accident_df_filter_make.fillna({'Make': 'Unknown'})

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [366]:
accident_df_filter_make['Model'].isna().sum()

np.int64(65)

In [367]:
# Remove null models
accident_df_filter_model = accident_df_filter_make.copy()
accident_df_filter_model = accident_df_filter_model.loc[~accident_df_filter_model['Model'].isna()]

In [368]:
# Find how many unique makes each model has
accident_df_filter_model[['Make', 'Model']].drop_duplicates()['Model'].value_counts()

Model
500              7
269C             4
7AC              4
100              4
G-164B           4
                ..
BH-47G3B2        1
PA28-201T        1
210-M            1
S1T              1
AG-CAT G-164A    1
Name: count, Length: 5682, dtype: int64

In [369]:
# Review a couple of the repeated makes to confirm these aren't missed duplicate makers
accident_df_filter_model[accident_df_filter_model['Model'] == '500']['Make'].unique()

array(['AERO COMMANDER', 'CESSNA', 'NORTH AMERICAN', 'HUGHES',
       'MCDONNELL DOUGLAS', 'EMBRAER', 'ROCKWELL'], dtype=object)

In [370]:
accident_df_filter_model[accident_df_filter_model['Model'] == 'G-164A']['Make'].unique()

array(['SCHWEIZER', 'GRUMMAN', 'GRUMMAN-SCHWEIZER', 'AIR TRACTOR'],
      dtype=object)

In [371]:
accident_df_filter_model['Make_Model_ID'] = accident_df_filter_model['Make'] + ' -- ' + accident_df_filter_model['Model']

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [372]:
accident_df_filter_other = accident_df_filter_model.copy()

In [373]:
# accident_df_filter_model[['Engine.Type', 'Weather.Condition', 'Number.of.Engines', 'Purpose.of.flight', 'Broad.phase.of.flight']].head(15)
accident_df_filter_other['Engine.Type'].unique()

array(['Reciprocating', 'Turbo Fan', 'Turbo Shaft', 'Turbo Prop',
       'Unknown', 'Turbo Jet', nan, 'Geared Turbofan', 'LR', 'UNK',
       'NONE'], dtype=object)

In [374]:
# Several variations of unknown and none
accident_df_filter_other.loc[accident_df_filter_other['Engine.Type'].isin(['UNK', 'NONE']), 'Engine.Type'] = 'Unknown'
accident_df_filter_other = accident_df_filter_other.fillna({'Engine.Type': 'Unknown'})

accident_df_filter_other['Engine.Type'].unique()

array(['Reciprocating', 'Turbo Fan', 'Turbo Shaft', 'Turbo Prop',
       'Unknown', 'Turbo Jet', 'Geared Turbofan', 'LR'], dtype=object)

In [375]:
accident_df_filter_other['Weather.Condition'].unique()

array(['IMC', 'UNK', 'VMC', nan, 'Unk'], dtype=object)

In [376]:
# Several variations of unknown
accident_df_filter_other.loc[accident_df_filter_other['Weather.Condition'] == 'Unk', 'Weather.Condition']  = 'UNK'
accident_df_filter_other = accident_df_filter_other.fillna({'Weather.Condition': 'UNK'})

accident_df_filter_other['Weather.Condition'].unique()

array(['IMC', 'UNK', 'VMC'], dtype=object)

In [377]:
# No cleaning needed
accident_df_filter_other['Number.of.Engines'].unique()

array([ 2.,  3.,  1., nan,  4.,  0.,  8.])

In [378]:
accident_df_filter_other['Purpose.of.flight'].unique()

array(['Personal', 'Unknown', 'Business', 'Ferry', 'Instructional',
       'Positioning', 'Aerial Application', 'Other Work Use',
       'Aerial Observation', 'Executive/corporate', nan,
       'Public Aircraft', 'Skydiving', 'Flight Test', 'Air Race/show',
       'Air Drop', 'Public Aircraft - Federal', 'Glider Tow',
       'Public Aircraft - Local', 'External Load',
       'Public Aircraft - State', 'Banner Tow', 'Firefighting',
       'Air Race show', 'PUBS', 'ASHO', 'PUBL'], dtype=object)

In [379]:
accident_df_filter_other = accident_df_filter_other.fillna({'Purpose.of.flight': 'Unknown'})

In [380]:
accident_df_filter_other['Broad.phase.of.flight'].unique()

array(['Approach', 'Cruise', nan, 'Maneuvering', 'Landing', 'Climb',
       'Standing', 'Takeoff', 'Taxi', 'Descent', 'Go-around', 'Unknown',
       'Other'], dtype=object)

In [381]:
accident_df_filter_other = accident_df_filter_other.fillna({'Broad.phase.of.flight': 'Unknown'})

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [382]:
columns = list(accident_df_filter_other.columns)
row_count = len(accident_df_filter_other)
columns_to_drop = ['Event.Date'] # Included because we have already made Event_date with a datetime object

# Find columns with greater than 50% null rate
for column in columns:
    if (accident_df_filter_other[column].isna().sum())/row_count > .5:
        columns_to_drop.append(column)

columns_to_drop

['Event.Date',
 'Latitude',
 'Longitude',
 'Aircraft.Category',
 'FAR.Description',
 'Schedule',
 'Air.carrier']

In [383]:
accident_df_filtered = accident_df_filter_other.drop(columns_to_drop, axis=1)

In [384]:
accident_df_filtered.head()

,Event.Id,Investigation.Type,Accident.Number,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Event_Date,Total_Passengers,Severe_Injury_Rate,Make_Model_ID
10613,20001214X35598,Accident,NYC85FNC02,"HOPKINTON, NH",United States,LEB,"LEBANON, NH",Non-Fatal,Destroyed,N47364,...,0.0,0.0,IMC,Approach,Probable Cause,NaN,1985-01-01,2.0,1.0,PIPER -- PA-34-200T
10614,20001214X35495,Accident,DEN85FA055,"EDGEWOOD, NM",United States,NaN,NaN,Fatal(2),Destroyed,N5069Q,...,0.0,0.0,IMC,Cruise,Probable Cause,NaN,1985-01-01,2.0,1.0,CESSNA -- 310N
10615,20001214X35494,Accident,DCA85RA007,"LA PAZ, Bolivia",Bolivia,LPB,LA PAZ INTERNATIONAL,Fatal(29),Destroyed,N819EA,...,0.0,0.0,UNK,Unknown,Foreign,06-02-1995,1985-01-01,29.0,1.0,BOEING -- 727-225
10617,20001214X35463,Accident,ATL85LA075,"DUBLIN, NC",United States,NaN,NaN,Non-Fatal,Substantial,N50913,...,0.0,1.0,VMC,Cruise,Probable Cause,NaN,1985-01-01,1.0,0.0,CESSNA -- 150J
10618,20001214X35584,Accident,MKC85FA042,"PAWNEE CITY, NE",United States,NaN,NaN,Fatal(1),Substantial,N7138F,...,0.0,0.0,VMC,Maneuvering,Probable Cause,NaN,1985-01-02,2.0,1.0,CESSNA -- 150F


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [385]:
accident_df_filtered.to_csv('data/AviationData_cleaned.csv')